<a href="https://colab.research.google.com/github/CALDISS-AAU/sdsphd19_coursematerials/blob/master/notebooks/Model_Explainability_Gensim_and_Sklearn_%F0%9F%92%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -qq -U gensim

!wget -O imdb.zip -qq "http://sds-datacrunch.aau.dk/public/imdb.zip"
!unzip imdb.zip


!pip -q install eli5

Archive:  imdb.zip
  inflating: test.tsv                
  inflating: train.tsv               
     |████████████████████████████████| 112kB 2.7MB/s 


In [24]:
# Read in the files and quickly print the size of the training and test set.

import pandas as pd
import numpy as np

train_df = pd.read_csv("train.tsv", sep="\t")
test_df = pd.read_csv("test.tsv", sep="\t")

print(f'Train size = {len(train_df)}')
print(f'Test size = {len(test_df)}')

Train size = 25000
Test size = 25000


In [25]:
# some basic text cleaning, removing HTML fragments (only a problem here)

import re

pattern = re.compile('<br /><br />')

print(train_df['review'].iloc[3])
print(pattern.subn(' ', train_df['review'].iloc[3])[0])

Spoilers ahead if you want to call them that...<br /><br />I would almost recommend this film just so people can truly see a 1/10. Where to begin, we'll start from the top...<br /><br />THE STORY: Don't believe the premise - the movie has nothing to do with abandoned cars, and people finially understanding what the mysterious happenings are. It's a draub, basic, go to cabin movie with no intensity or "effort".<br /><br />THE SCREENPLAY: I usually give credit to indie screenwriters, it's hard work when you are starting out...but this is crap. The story is flat - it leaves you emotionless the entire movie. The dialogue is extremely weak and predictable boasting lines of "Woah, you totally freaked me out" and "I was wondering if you'd uh...if you'd like to..uh, would you come to the cabin with me?". It makes me want to rip out all my hair, one strand at a time and feed it to myself.<br /><br />THE CHARACTERS: HOLY CRAP!!!! Some have described the characters as flat, I want to take it one 

In [0]:
# application of the cleaning mask to everthing

train_df['review'] = train_df['review'].apply(lambda text: pattern.subn(' ', text)[0])
test_df['review'] = test_df['review'].apply(lambda text: pattern.subn(' ', text)[0])

In [27]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Import the dictionary builder
from gensim.corpora.dictionary import Dictionary

In [0]:
# Import stopwords

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [0]:
# Tokenize our texts and remove stopwords, also kick out numbers, lower everything

train_tokens = train_df['review'].map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
test_tokens = test_df['review'].map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
# Generate a dictionary
dictionary = Dictionary(train_tokens)

In [0]:
# Filter it for extreme stuff
dictionary.filter_extremes(no_below = 10, no_above=0.4)

In [0]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from gensim.sklearn_api import LsiTransformer, Text2BowTransformer, TfIdfTransformer
import numpy as np

# Create stages for our pipeline (including gensim and sklearn models alike).
bow = Text2BowTransformer(prune_at = 5000)
tfidf = TfIdfTransformer(dictionary=dictionary) # Here you can add a custom fancy tokenizer in which you define more preprocessing...
model = LsiTransformer(num_topics=300)
clf = XGBClassifier()
pipe = Pipeline([('BOW', bow), ('TFIDF', tfidf), ('features', model), ('classifier', clf)])

# How well does our pipeline perform on the training set?
score = pipe.fit(train_df['review'][:5000], train_df.is_positive[:5000]).score(test_df['review'][:5000], test_df.is_positive[:5000])

In [59]:
score

0.7418

In [62]:
doc = test_df['review'][0]
print(doc)

A formulaic story with all the tired cliches. I was shocked that the horrible script became a movie! What a waste! How many ORIGINAL scripts are scattered around without being produced?   At the climax I could predict what will happen in every single shot.  Oh, and don't even get me started about the idiotic sexual tension between the gorgeous female cop and the paralyzed cop who can barely move a finger. YEAH RIGHT. I CAN BUY THAT. What next? Fingers porn?


In [63]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction()